In [1]:
import findspark
findspark.init()

import sys
from pyspark import SparkContext, SparkConf
from scipy import spatial
import numpy as np

In [2]:
# use a function to load the movie names corresponding to the movie IDs
def loadMovieNames():
    movieNames = {}
    with open('ml-100k\u.item') as file:
        for line in file:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1].decode('ascii','ignore')
            
    return movieNames

In [3]:
# This time we will use a slightly different similarity function. Specifically our function will still be the cosine similarity
# metric but will be weighted somehow by the number of ratings available. More the number of ratings, higher the overall 
# similarity score

def filterDuplicates((userid, pairs)):
    movie1 = pairs[0][0]
    movie2 = pairs[1][0]
    return movie1 < movie2

# We will pass the number of ratings through a sigmoid function which essentially squashes input between 0 to 1. Higher the 
# number of ratings, closer is the output of the sigmoid function to 1. Movies with very low number of ratings will have the 
# sigmoid output close to 0.5. We also choose a rise factor (< 1) that governs how fast the sigmoid saturates to 1.

def sigmoid(x,a):
    return float(1/(1+ np.exp(-a*x)))


# our net similarity is a weighted sum of the cosine similarity and the output of the sigmoid function. The weight is called 
# alpha
def similarityFunc(list_of_ratings):
    arr = np.array(list_of_ratings)
    # the method gives the distance. 1 - the value is the similarity
    sim = 1-spatial.distance.cosine(arr[:,0], arr[:,1])
    num_of_pairs = len(list_of_ratings)
    weight = sigmoid(num_of_pairs, 0.005) # rise factor of 0.005, essentially means a much slower rise to 1 than standard sigmoid
    alpha = 0.5 
    weighted_sim = alpha*weight + (1-alpha)*sim
    
    return (float(weighted_sim), num_of_pairs)
    
    
def makeItemPairs((userid, pairs)):
    (movie1, rating1) = pairs[0]
    (movie2, rating2) = pairs[1]
    
    return ((movie1,movie2),(rating1,rating2))

In [4]:
# the dataset has 100,000 records. So we will be using all the cores available in the computer
# by setting the setMaster argument to 'local[*]'

conf = SparkConf().setMaster('local[*]').setAppName('MovieSimi_ver2')
sc = SparkContext(conf = conf)

# load the movie names dataset into a large dictionary called names
names = loadMovieNames()

data = sc.textFile('.\ml-100k\u.data')

# map ratings in the form of userid as key and (movieid, rating) as value
user_ratings = data.map(lambda x: x.split('\t')).map(lambda parts: (int(parts[0]), (int(parts[1]), float(parts[2]))))

# compute a list of movie pairs with rating pairs from all users who have rated that pair of movies
# this will have each record in the form userid as key and  ((movieid1, rating1), (movieid2, rating2)) as the value
movie_ratings = user_ratings.join(user_ratings)

# removing duplicates...since a join could result in both movies being same and all combinations of the same movie pair in 
# different orders. We will only keep the movies where the first id is less than the second id. 
movie_ratings_clean = movie_ratings.filter(filterDuplicates)

# the records are still in the form userid as key and  ((movieid1, rating1), (movieid2, rating2)) as the value
# convert this rdd to the form (movieid1, movieid2) as the key and (rating1,rating2) as the value

movie_pairs = movie_ratings_clean.map(makeItemPairs)

# Now we will group by movie pairs to find all available pairs of ratings for each unique movie pair
movie_pairs_group = movie_pairs.groupByKey().map(lambda (x,y) : (x, list(y)))

#now for each movie pair as the key the value is a list of rating pairs collected from all users. We will consider each rating
# pair as elements from two vectors and essentially compute the similarity of the two vectors, each vector being the collection
# of ratings

movie_pair_simi = movie_pairs_group.mapValues(similarityFunc).persist() #we cache this rdd as this will be used later

# each record is now of the form (movieid1, movieid2) as key and (similarity, number of rating pairs) as the value
print movie_pair_simi.take(10)

[((197, 1097), (0.7423110081201001, 7)), ((42, 364), (0.7159167404158419, 18)), ((773, 1409), (0.7506249986979199, 1)), ((273, 617), (0.7370222333904755, 7)), ((372, 974), (0.7506249986979199, 1)), ((789, 865), (0.7467487273331919, 3)), ((496, 1314), (0.7407083328480895, 4)), ((246, 1008), (0.7337086520601352, 18)), ((856, 1006), (0.7405811481955664, 10)), ((747, 795), (0.6623528535395095, 6))]


In [5]:
# now we will get the similar movies for the user provided movie id
# Let's choose movie ID 50 which is a Star Wars movie

user_choice = 50
num_of_reco = 20 # number of similar movie suggestions to be sent to the output 

num_of_ratings = 50 # to select the similar movies which have atleast this number of ratings
    
# filter the movies which satisfy the given criteria

results = movie_pair_simi.filter( lambda (pair, (sim,num)) : (user_choice in  pair) and (num >= num_of_ratings))
results_final  = results.map(lambda (x,y) : (y,x)).sortByKey(ascending = False).take(num_of_reco)


print 'Top {} similar movies for the movie:  {}\n'.format(num_of_reco, names[user_choice])
for val in results_final:
    ((sim,num),pair) = val
    similarMovies = pair[0]
    if similarMovies==user_choice:
        similarMovies=  pair[1]
        
    print names[similarMovies] + ' with score: {:.2} from {} ratings'.format(sim, num)  

Top 20 similar movies for the movie:  Star Wars (1977)

Return of the Jedi (1983) with score: 0.95 from 480 ratings
Raiders of the Lost Ark (1981) with score: 0.93 from 380 ratings
Empire Strikes Back, The (1980) with score: 0.92 from 345 ratings
Toy Story (1995) with score: 0.92 from 381 ratings
Fargo (1996) with score: 0.92 from 394 ratings
Godfather, The (1972) with score: 0.91 from 357 ratings
Independence Day (ID4) (1996) with score: 0.9 from 362 ratings
Silence of the Lambs, The (1991) with score: 0.9 from 335 ratings
Contact (1997) with score: 0.9 from 334 ratings
Star Trek: First Contact (1996) with score: 0.9 from 316 ratings
Indiana Jones and the Last Crusade (1989) with score: 0.9 from 304 ratings
Twelve Monkeys (1995) with score: 0.9 from 324 ratings
Back to the Future (1985) with score: 0.9 from 309 ratings
Fugitive, The (1993) with score: 0.89 from 297 ratings
Pulp Fiction (1994) with score: 0.89 from 330 ratings
Rock, The (1996) with score: 0.89 from 312 ratings
Princess

In [ ]:
# The modified weighted similarity metric does slightly better. For example we see a 'Star Trek' and 'Terminator' movie included
# in the suggestions which are clearly science fiction and better recommendations than movies like "Glory" and "Usual Suspects" 
# both of which are now absent. Still we can do better. In the next version we will incorporate genre information alongwith 
# the weighted similarity measure to see how the results differ.